In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

In [4]:
imp = pd.read_csv('/Users/alecasillas/Desktop/COURSES/STA4930/datasets/imputed.csv', encoding= 'utf-8')

In [5]:
# Load dataset
# Replace 'imp.csv' with the actual dataset file path
df = imp

# Define features and target
features = ["e_gdppc", "v2eldonate", "v2elpubfin", "v2elembaut", "v2elrgpwr", "v2ellocpwr", "v2psbars", "v2exrescon", "v2cltort", "v2caautmob"]
target = "rescaled"

# Drop rows with missing values
df = df.dropna(subset=features + [target])

# Train a model
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

# Function to filter dataset based on selected country and year
def filter_data(country, year):
    filtered = df[(df['country_name'] == country) & (df['year'] == year)]
    return filtered

# Function to predict rescaled score based on inputs
def predict_rescaled(feature_values):
    feature_array = np.array([feature_values]).reshape(1, -1)
    return model.predict(feature_array)[0]

# Interactive function
def simulate(country, year, **kwargs):
    data = filter_data(country, year)
    
    if data.empty:
        display("No data available for the selected country and year.")
        return
    
    original_rescaled = data[target].values[0] if target in data.columns else None
    
    # Predict rescaled score based on user inputs
    feature_values = [kwargs[feature] for feature in features]
    predicted_rescaled = predict_rescaled(feature_values)
    
    display(f"Original Rescaled Score: {original_rescaled}")
    display(f"Predicted Rescaled Score: {predicted_rescaled}")

# Dropdowns for country and year
country_dropdown = widgets.Dropdown(options=df['country_name'].unique(), description='Country')
year_dropdown = widgets.IntSlider(min=df['year'].min(), max=df['year'].max(), step=1, description='Year')

# Sliders for feature adjustments
def create_feature_sliders(country, year):
    data = filter_data(country, year)
    sliders = {}
    for feature in features:
        default_value = data[feature].values[0] if not data.empty and feature in data.columns and pd.notna(data[feature].values[0]) else 0.0
        sliders[feature] = widgets.FloatSlider(min=-1, max=1, step=0.1, value=default_value, description=feature)
    return sliders

def update_sliders(*args):
    global feature_sliders
    feature_sliders = create_feature_sliders(country_dropdown.value, year_dropdown.value)
    display(widgets.VBox(list(feature_sliders.values())))
    update_simulation(country_dropdown.value, year_dropdown.value, **{f: s.value for f, s in feature_sliders.items()})

feature_sliders = create_feature_sliders(country_dropdown.value, year_dropdown.value)

def update_simulation(country, year, **kwargs):
    simulate(country, year, **kwargs)



In [6]:
# Observe changes in country and year
year_dropdown.observe(update_sliders, names='value')
country_dropdown.observe(update_sliders, names='value')

#display(widgets.VBox([country_dropdown, year_dropdown] + list(feature_sliders.values())))
widgets.interactive(update_simulation, country=country_dropdown, year=year_dropdown, **feature_sliders)

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Angola', 'Albania', 'United Ara…